# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [31]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [33]:
# Use this cell for any import statements you add
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
artists = {'Kendrick Lamar':"https://www.azlyrics.com/k/kendricklamar.html",
           'Hozier':"https://www.azlyrics.com/h/hozier.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: The robots.txt file tells us that we are not allowed to access the /lyricsdb/ directory or the /song/ directory, but are allowed to access all other directories. For the purposes of this assignment, we are accessing /lyrics/, so yes, this is allowed by the page.


In [19]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # create an empty list for the links
    link_list = []
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/lyrics/'):
            link_list.append(link['href'])
            
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    lyrics_pages[artist].extend(link_list)

Let's make sure we have enough lyrics pages to scrape. 

In [21]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [22]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For Kendrick Lamar we have 186.
The full pull will take for this artist will take 0.52 hours.
For Hozier we have 74.
The full pull will take for this artist will take 0.21 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [28]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [34]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [55]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist, artist_page in lyrics_pages.items():
    
    # 1. Build a subfolder for the artist
    if os.path.isdir(f"lyrics/{artist}"): 
        shutil.rmtree(f"lyrics/{artist}/")

    os.makedirs(f"lyrics/{artist}")
    
    # 2. Iterate over the lyrics pages
    for link in artist_page:
        # 3. Request the lyrics page. 
        r = requests.get(url_stub + link)
        time.sleep(5 + 10*random.random())

        # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.content, 'html.parser')

        title = soup.title.string

        lyrics_div = soup.find('div', class_='col-xs-12 col-lg-8 text-center')
        lyrics = lyrics_div.get_text(separator='\n', strip=True)

        # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
        #    to generate the filename. 
        filename = generate_filename_from_link(link)

        with open(filename, 'w', encoding='utf-8') as file:
            # write to file
            file.write(title + '\n\n')
            file.write(lyrics)


Kendrick Lamar - Is It Love Lyrics | AZLyrics.com
"Is It Love" lyrics
Kendrick Lamar Lyrics
"Is It Love"
(feat. Angela McCluskey)
[Angela McCluskey (Kendrick Lamar):]
Sit still and close your eyes (smoke to it)
What's behind the other door? Oh-ohh
No more silence (no more silence)
Don't kill this thing we got called love (don't shoot)
Just searching for the perfect shot
[Angela McCluskey:]
When love comes calling, don't look back
When love comes calling, don't look away
When love comes calling, don't look back
When love comes calling, don't look away
[Kendrick Lamar (Angela McCluskey):]
I used to write rhymes all day and all night
When y'all was playing PlayStation, my pencil was erasing lines
My conscience only knew what's half-tight
At 3:14, it's time to get me a slice my nigga
This is a dog's fight my nigga
The soundtrack to life my nigga
Kendrick Lamar, his momma called him that
He watched House Party and ate Apple Jacks
Sold Sega games, his cousin sold crack
He pumped Reeboks, his

Kendrick Lamar - P & P Lyrics | AZLyrics.com
"P & P" lyrics
Kendrick Lamar Lyrics
"P & P"
(feat. Ab Soul)
Oh what up, ho? Oh, what up?
I said oh, what up, ho? Oh, what up?
I said oh, what up, ho? Oh, what up?
Well, alright
[Kendrick Lamar:]
I'm going through something with life
But pussy and Patrón make you feel alright
Pussy and Patrón make you feel alright
Pussy and Patrón, that's some great advice
I'm going through something with life
But pussy and Patrón make you feel alright
Pussy and Patrón make you feel alright
Pussy and Patrón, that's some great advice
Welcome to my diary, stressing got me gray hairs
Something to inspire me, rather than society's
Woes, let me go, let me shine a lil' bit love
I want diamonds too, Ronnie on Player's Club
I used to have a 9 to 5, fresh out of school that was '05
That bitch was racist, got me fired, ever since then I had no job
Pushing in my mama van, stop for gas on Rosecrans
Trust me these niggas rushed me for something my cousin probably did
Gui

Kendrick Lamar - I Am (Interlude) Lyrics | AZLyrics.com
"I Am (Interlude)" lyrics
Kendrick Lamar Lyrics
"I Am (Interlude)"
See y'all don't understand me
My plan B is to win ya hearts before I win a Grammy
Kendrick Lamar, words like a sword in the hands of a Spartan
Marching on Rosecrans in Compton
You can't 'cause conflict or corrupt my spirit
I'm on point like period
I've been there and gone before you get near it
Ooh wee, that boy remind me of a young Martin Luther
The way he piece up troopers, then round up shooters
Like Malcolm X did, I stand for what I believe in
Family, God and honor
From Chicago, my daddy and my mamma
Came to Compton to accomplish one thing
Raise a king, reign supreme, named Kendrick
I ain't lying, it stand for king and I am one
My unborn son and grandson will live royal
From the coochie to the soil
The meek shall inherit the Earth
Well I've had this world since birth
Feel the good kid's hunger
The Crips made strong, the Pirus made stronger
Muscle in my heart
St

Kendrick Lamar - I Do This Lyrics | AZLyrics.com
"I Do This" lyrics
Kendrick Lamar Lyrics
"I Do This"
(feat. Jay Rock)
[Kendrick Lamar:]
The homies say I'm the truest
The bitches think I'm the cutest
The definition what cool is
Boy, I thought you knew this
Bitch, I do this!
Uh, I used to wanna be like Michael Jordan
Figured that I hit the NBA and make a fortune
Thank God for these rap recordings
I can ball like him on every verse and chorus
501s, Louis Letterman jacket
Little brother only 5, mamma put him on my taxes
You know that Compton ghetto shit
That every section 8 tenant is familiar with
Ooh wee, they say he something like the 70s
But I'm from '87, with a thousand dimes
You got 80 sevens
Gourmet sneakers, hair nappy
Fucking all my high school teachers
That was '05 bro
And I don't like 'em if the ass thin
And I don't wife 'em if a nigga is her best friend
The fuck I look like?
A million bucks, and that's why
The homies say I'm the truest
The bitches think I'm the cutest
The defin

Kendrick Lamar - Faith Lyrics | AZLyrics.com
"Faith" lyrics
Kendrick Lamar Lyrics
"Faith"
(feat. BJ the Chicago Kid, Punch)
[Kendrick Lamar:]
Cloud nine, Kendrick Lamar, uh
I take a sip of Hennessy and then get pissy drunk
I ain't a drinker, I'm a thinker, call it what you want
But if you turn your back, know that you just missed your chance
To witness the realest shit that's ever been told to man
I found myself losing focus at a Sunday service
Embarrassed so I start questioning God, "What is my purpose?"
He said to live the way he did, that's all he want from me
Spread the word and witness, he rose on the first Sunday
I said alright, enthused that my Lord gave a listen
I opened my Bible in search to be a better Christian
And this from a person that never believed in religion
But shit, my life is so fucked up, man; I can't help but give in
I'm giving testimonies to strangers I never met
Hopped on the pulpit and told them how I was truly blessed
Felt like I'm free from all my sins when 

...

Hozier - Blood Lyrics | AZLyrics.com
"Blood" lyrics
Hozier Lyrics
"Blood"
(live version)
Tryin' hard to recognise
Some pure motive inside of me
A creature that would horrify
Any child that I used to be
Oh, give no faith to show
Started to smile, so I showed my teeth
No more than flesh and bone
Doin' so much just to watch someone bleed
But I love the very blood of you
It keeps its heat in spite of you
Oh, the heart that beats
To keep you here with me always
Tryin' hard, I nearly find
Innocence in spite of me
Oh, creature that would terrify
Any child left inside of me
Oh, that's a cold insight
Nothin' above us and nothin' below
Ahh, but you might be right
If there is no heaven and there is no soul
I just love the very blood of you
It keeps its heat in spite of you
Oh, the heart that beats
To keep you here with me always
Always...
Oh, I love the very blood of you
It keeps its heat in spite of you
Oh, the heart that beats
To keep you here with me always
Oh, I love the very blood of you
Lea

Hozier - Lay Me Down Lyrics | AZLyrics.com
"Lay Me Down" lyrics
Hozier Lyrics
"Lay Me Down"
(originally by Sam Smith)
Yes, I do, I believe
That one day I will be
Where I was right there
Right next to you
And it's hard
These days just seem so dark
The moon and the stars
Are nothing without you
Your touch, your skin
Where do I begin?
No words can explain
The way I'm missing you
Deny this emptiness
This hole that I'm inside
These tears
They tell their own story
Told me not to cry when you were gone
Enough with this feeling, it's much too strong
Can I lay by your side?
Next to you
Next to you
And make sure you're alright
I'll take care of you
I don't want to be here if I can't be with you tonight
I'm reaching out to you
Can you hear my call?
This hurt that I've been through
I'm missing you like crazy, like crazy, like crazy
You told me not to cry when you were gone
Enough with this feeling, it's much too strong
Can I lay by your side?
Next to you
Next to you
And make sure you're alright
I'

Hozier - Sorry Not Sorry Lyrics | AZLyrics.com
"Sorry Not Sorry" lyrics
Hozier Lyrics
"Sorry Not Sorry"
(originally by Demi Lovato)
I'm out here lookin' like revenge
Feelin' like a ten, the best I've ever been
And, yeah, I know how bad it must hurt to see me like this
But it gets worse
Now, you're out here lookin' like regret, ain't too proud to beg
Second chance, you'll never get
And, yeah, I know how bad it must hurt to see me like this
But it gets worse
(Payback is a bad)
(And baby, I'm the baddest)
(You with a savage)
Can't have this, can't have this, ah
And it'd be nice of me to take it easy on ya
Nah
Baby, I'm sorry (Not sorry)
Baby, I'm sorry (Not sorry)
Bein' so bad got me feelin' so good
Showin' you up like I knew that I would
Baby, I'm sorry (Not sorry)
Baby, I'm sorry (Not sorry)
Feelin' inspired 'cause the tables have turned
Yeah, I'm on fire and I know that this burns
Baby, fineness is the way to kill
Tell me how it feels, bet it's such a bitter pill
And yeah I know, you t

Hozier - The Wages Lyrics | AZLyrics.com
"The Wages" lyrics
Hozier Lyrics
"The Wages"
Although many contend
You may find hope ascend
To mark just how everything changes
Whether the temperature highs
Or the sea level rising
It's all going up but the wages
And murdered reporters
And talk of hard borders
Sky-bound as a heavenly choir
Or the stock price in Boeing
The screams of the dying
Or the skies over Yemen last night
It's all going up but the wages
And maybe it's time
It's all going up but the wages
And maybe it's time
Don't mention the rent
Or the money that's spent
Keeping the banks in good health
Or ridiculous claims
Or the level of blame
Pointed to anything else
And preachy musicians
And those stuck in prison
With scarcely enough for a blunt
Or the pressure of blood
That new hotel in the hood
With an oligarch's name on the front
It's all going up but the wages
And maybe it's time
It's all going up but the wages
And maybe it's time
It's all going up with the bump in the heating bil

AttributeError: 'NoneType' object has no attribute 'get_text'

In [56]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.8 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [57]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [58]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For Hozier we have 0 files.
For Hozier we have roughly 0 words, 0 are unique.
For Kendrick Lamar we have 0 files.
For Kendrick Lamar we have roughly 0 words, 0 are unique.
